In [2]:
import yaml

data = {
    "train": "data/images/train",
    "val": "data/images/val",
    "nc": 1 ,
    "names":  ["car"]
}

with open("dataset.yaml", "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("dataset.yaml créé :", "dataset.yaml")


dataset.yaml créé : dataset.yaml


# Quantization 

In [3]:
# Train
!pip install ultralytics
!pip install onnxruntime
!pip install -U huggingface_hub
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from onnxruntime.quantization import quantize_dynamic, QuantType, quantize_static
from onnxruntime.quantization import CalibrationDataReader,QuantFormat, CalibrationMethod

Ultralytics 8.4.10 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6634.2/8062.4 GB disk)


In [4]:
model_path = "/kaggle/input/z/other/default/1/best (4).pt"

import shutil

src = "/kaggle/input/model/other/default/1/best (4).pt"
dst = "/kaggle/working/best.pt"

shutil.copy(src, dst)

print("Copié vers :", dst)


Copié vers : /kaggle/working/best.pt


In [5]:
model = YOLO("/kaggle/working/best.pt")

In [6]:
model.export(
    format="onnx",
    #half=True,
    imgsz=768,
    #save_dir="/kaggle/working"
)

Ultralytics 8.4.10 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs

PyTorch: starting from '/kaggle/working/best.pt' with input shape (1, 3, 768, 768) BCHW and output shape(s) (1, 5, 12096) (18.0 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 230ms
Prepared 2 packages in 2.69s
Installed 2 packages in 101ms
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 3.4s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.20.1 opset 22...


/usr/local/lib/python3.12/dist-packages/torch/onnx/utils.py:1397: OnnxExporterWarning: Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(


ONNX: slimming with onnxslim 0.1.82...
ONNX: export success ✅ 6.3s, saved as '/kaggle/working/best.onnx' (35.6 MB)

Export complete (7.9s)
Results saved to /kaggle/working
Predict:         yolo predict task=detect model=/kaggle/working/best.onnx imgsz=768 
Validate:        yolo val task=detect model=/kaggle/working/best.onnx imgsz=768 data=dataset.yaml  
Visualize:       https://netron.app


'/kaggle/working/best.onnx'

In [7]:
!python -m onnxruntime.quantization.preprocess --input "best.onnx" --output preprocessed.onnx

# Dynamic Quantization

In [8]:
model_fp32 = 'preprocessed.onnx'
model_int8 = 'dynamic_quantizedS.onnx'

# Quantize 
quantize_dynamic(model_fp32, 
                 model_int8, 
                 weight_type=QuantType.QUInt8)


In [9]:
from ultralytics import YOLO

# Charger le modèle ONNX avec la tâche spécifiée
model = YOLO("dynamic_quantizedS.onnx", task="detect")

# Évaluer le modèle sur un jeu de données
metrics = model.val(data="dataset.yaml", 
                    split="val",  device="cuda",
                   imgsz=768,)  # Utiliser le GPU


Ultralytics 8.4.10 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Loading dynamic_quantizedS.onnx for ONNX Runtime inference...
WARNING ⚠️ CUDA requested but CUDAExecutionProvider not available. Using CPU...
Using ONNX Runtime 1.23.2 with CPUExecutionProvider
Setting batch=1 input of shape (1, 3, 768, 768)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1560.3±595.5 MB/s, size: 76.9 KB)
val: Scanning /kaggle/working/data/labels/val... 11231 images, 35 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 11231/11231 1.4Kit/s 8.1s
val: New cache created: /kaggle/working/data/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11231/11231 2.4it/s 1:17:35
                   all      11231     117064      0.794      0.799      0.845      0.643
Speed: 1.4ms preprocess, 402.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /kaggle/working/runs/detect/val


# Static Quantization

In [10]:
import os
import random

class ImageCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.idx = 0
        self.input_name = "images"

    def preprocess(self, frame):
        # Same preprocessing that you do before feeding it to the model
        frame = cv2.imread(frame)
        X = cv2.resize(frame, (768, 768))
        image_data = np.array(X).astype(np.float32) / 255.0  # Normalize to [0, 1] range
        image_data = np.transpose(image_data, (2, 0, 1))  # (H, W, C) -> (C, H, W)
        image_data = np.expand_dims(image_data, axis=0)  # Add batch dimension
        return image_data

    def get_next(self):
        # method to iterate through the data set
        if self.idx >= len(self.image_paths):
            return None

        image_path = self.image_paths[self.idx]
        input_data = self.preprocess(image_path)
        self.idx += 1
        return {self.input_name: input_data}


# Spécifiez le chemin de votre dossier
path = "/kaggle/working/data/images/val"

# Récupérez tous les fichiers d'image dans ce dossier
calibration_image_paths = [os.path.join(path, filename) for filename in os.listdir(path) if filename.lower().endswith(('.jpg', '.jpeg', '.png'))]

random.seed(42)  # pour reproductibilité IEEE

random.shuffle(calibration_image_paths)

calibration_data_reader = ImageCalibrationDataReader(
    calibration_image_paths[:1000]
)

In [11]:
import numpy as np
import xml.etree.ElementTree as ET
import shutil
from glob import glob
import os
import cv2

quantize_static('preprocessed.onnx', "static_quantized.onnx",
                weight_type=QuantType.QInt8,
                activation_type=QuantType.QUInt8,
                calibration_data_reader=calibration_data_reader,
                quant_format=QuantFormat.QDQ,
                nodes_to_exclude=['/model.21/Concat', '/model.21/dfl/Reshape', '/model.21/dfl/Transpose',
                                 '/model.21/dfl/Softmax',  
                                 '/model.21/dfl/conv/Conv', '/model.21/dfl/Reshape_1', '/model.21/Slice_1',
                                 '/model.21/Slice', '/model.21/Add_1', '/model.21/Sub','/model.21/Add_2',
                                  '/model.21/Sub_1','/model.21/Div_1',
                                 '/model.21/Concat_2', '/model.21/Mul_2', '/model.21/Concat_3'],
                per_channel=True,
                reduce_range=True  ,
                )  

In [12]:
model = YOLO("static_quantized.onnx")
metrics = model.val(data="dataset.yaml", 
                    split="val",  device="cpu"
                    , imgsz=768, conf=0.4
                   )

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics 8.4.10 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Loading static_quantized.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 with CPUExecutionProvider
Setting batch=1 input of shape (1, 3, 768, 768)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1520.5±547.8 MB/s, size: 77.5 KB)
val: Scanning /kaggle/working/data/labels/val.cache... 11231 images, 35 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 11231/11231 3.9Git/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11231/11231 3.6it/s 51:47
                   all      11231     117064      0.809      0.786      0.834      0.665
Speed: 1.5ms preprocess, 264.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /kaggle/working/runs/detect/val2